In [21]:
# Al quitar la tabla del notebook, me ha desaparecido el contenido,la he vuelto a pegar. Tenemos que volcarla en la base de datos.

tabla_suministros = '''
CREATE TABLE Registro(
FECHA    DATE, 
ID_PROVEEDOR    INT,
ID_PIEZA    INT,
CANTIDAD    INT     
)
'''
cursor.execute(tabla_suministros)

query = "SELECT * FROM sqlite_master WHERE type == 'table'"
cursor.execute(query)
cursor.fetchall()

[('table',
  'Pieza',
  'Pieza',
  2,
  'CREATE TABLE Pieza (\n            ID INTEGER PRIMARY KEY AUTOINCREMENT,\n            Nombre TEXT NOT NULL,\n            Color TEXT,\n            Precio REAL,\n            ID_Categoria INTEGER\n        )'),
 ('table',
  'sqlite_sequence',
  'sqlite_sequence',
  3,
  'CREATE TABLE sqlite_sequence(name,seq)'),
 ('table',
  'Categoria',
  'Categoria',
  4,
  'CREATE TABLE Categoria (\n            ID INTEGER PRIMARY KEY AUTOINCREMENT,\n            Nombre TEXT NOT NULL UNIQUE\n        )'),
 ('table',
  'Proveedores',
  'Proveedores',
  6,
  'CREATE TABLE Proveedores (\n            ID INTEGER PRIMARY KEY AUTOINCREMENT,\n            Nombre TEXT NOT NULL,\n            Direccion TEXT,\n            Ciudad TEXT,\n            Provincia TEXT,\n            Codigo TEXT UNIQUE,\n            Fecha_Registro DATE DEFAULT CURRENT_DATE\n        )'),
 ('table',
  'Registro',
  'Registro',
  8,
  'CREATE TABLE Registro(\nFECHA    DATE, \nID_PROVEEDOR    INT,\nID_PIEZA 

In [1]:
import datetime as dt
import pandas as pd
import sqlite3

conexion = sqlite3.connect("./piezas.db")
cursor = conexion.cursor()

In [2]:
def consulta_elemento(tabla, nombre):
    query = f"""
        SELECT *
        FROM {tabla}
        WHERE Nombre LIKE '%{nombre}%'
        """
    ID = pd.read_sql(query, conexion).ID

    if len(ID) == 0:
        return 'No se encontró ID.'
    return ID

def consulta_id(tabla, ID:list):
    query = f"""
        SELECT *
        FROM {tabla}
        WHERE ID = '{ID}'
        """
    return pd.read_sql(query, conexion)['Nombre'][0]

def consulta(suministro):
    tabla, algo = suministro[0], suministro[1]

    elemento = input(f'Inserte {algo}:').capitalize()
    resultados = consulta_elemento(tabla, elemento)

    if type(resultados) == str:
        print(resultados)
        return 'salida'
    else:
        if len(resultados) > 1:
            for index, resultado in enumerate(resultados):
                print(index, consulta_id(tabla, resultado))

            index = input(f'Seleccione {algo}:')
            while index.isnumeric() != True or int(index) > len(resultados)-1 or int(len(resultados)-1) <0:
                index = input(f'Inserte un número entre 0-{len(resultados)-1}:')
            ID = resultados[int(index)]
        else:
            ID = resultados[0]
            index = 0
    
    print(consulta_id(tabla, ID))
    
    return str(ID)

def consulta_tabla(tabla):
    return pd.read_sql(f"SELECT * FROM {tabla}", conexion)

In [3]:
def insertar_suministros(fecha, ID_proveedor, ID_pieza, cantidad):
    query = f"INSERT INTO Registro (FECHA, ID_PROVEEDOR, ID_PIEZA, CANTIDAD) VALUES (?,?,?,?)"
    cursor.execute(query,(fecha,ID_proveedor,ID_pieza,cantidad))
    conexion.commit()

    query = """SELECT * FROM Registro"""
    df= pd.read_sql(query, conexion).set_index('FECHA')

    return df

In [4]:
def insertar_campo(algo):
    if algo == "proveedor":

        nombre = input("Introduzca el nombre del nuevo proveedor:")
        direccion = input("Introduzca la dirección, solo el tipo de vía y número:")
        ciudad = input("Introduzca la ciudad:")
        provincia = input("Introduzca la provincia:")
        ID_prov = input("Introduzca un nuevo ID:") # podemos hacer que detecte el indice + 1 del df?? si introducimos un num repetido, esto se rompe al ser el primary key

        query = f"INSERT INTO Proveedores (ID, NOMBRE, DIRECCION, CIUDAD, PROVINCIA) VALUES (?,?,?,?,?)"
        cursor.execute(query,(ID_prov, nombre, direccion, ciudad, provincia))
        conexion.commit()

        query = """SELECT * FROM Proveedores"""
        df= pd.read_sql(query, conexion).set_index('ID')
    else:
        nombre = input("Introduzca el nombre de la nueva pieza:")
        color = input("Introduzca el color:")
        precio = float(input("Introduzca el precio:"))
        ID_cat = "unknown"   # unkown ya que no tenemos funcion que consulte categorias
        ID_pieza = input("Introduzca un nuevo ID:") # lo mismo que con el ID proveedor
        
        query = f"INSERT INTO Pieza (ID, NOMBRE, COLOR, PRECIO, ID_CATEGORIA) VALUES (?,?,?,?,?)"
        cursor.execute(query,(ID_pieza, nombre, color, precio, ID_cat))
        conexion.commit()

        query = """SELECT * FROM Pieza"""
        df= pd.read_sql(query, conexion).set_index('ID')
        
    return df

In [11]:
def elige(algo, index):
    texto = [f'No se encontró {algo}, ¿desea agregar {algo} (S/N)', f'¿Desea añadir {algo}? (S/N)']
    eleccion = input(texto[index]).upper()
    while eleccion != 'S' and eleccion != 'N':
        eleccion = input(texto[index]).upper()
    return eleccion        

In [26]:
## MAIN ##

hoy = dt.datetime.now().strftime("%Y-%m-%d")

tablas = [('Proveedores','proveedor'),('Pieza','pieza')]

while True:
    ID_proveedor = consulta(tablas[0])
    if ID_proveedor == 'salida':
        if elige(tablas[0][1], 0) == 'S':
            print(insertar_campo(tablas[0][1]))
        else:
            break

    while True:
        ID_pieza = consulta(tablas[1])
        if ID_pieza == 'salida':
            if elige(tablas[1][1], 0) == 'S':
                print(insertar_campo(tablas[1][1]))
            else:
                break      
        cantidad = input(f'Indique cantidad de piezas:')
        while cantidad.isnumeric() != True or int(cantidad) <= 0:
            cantidad = input(f'Inserte una cantidad:')
        print(cantidad)
        
        insertar_suministros(hoy, ID_proveedor, ID_pieza, cantidad)
        print(consulta_tabla('Registro'))

        if elige(tablas[1][1], 1) == 'S':
            continue
        else:
            break
    if elige(tablas[0][1], 1) == 'S':
        continue
    break

0 Herramientas y Tornillos S.L.
1 Tornillones SA
Tornillones SA
Tornillo hexagonal
4
        FECHA  ID_PROVEEDOR  ID_PIEZA  CANTIDAD
0  2025-04-24             6         1         4
0 Correa dentada
1 Correa trapezoidal
Correa trapezoidal
6
        FECHA  ID_PROVEEDOR  ID_PIEZA  CANTIDAD
0  2025-04-24             6         1         4
1  2025-04-24             6        26         6
Suministros Industriales López
Junta tórica
10
        FECHA  ID_PROVEEDOR  ID_PIEZA  CANTIDAD
0  2025-04-24             6         1         4
1  2025-04-24             6        26         6
2  2025-04-24             2         5        10


In [ ]:
# sintaxis para borrar registros, se puede sacar una función de aquí también

query = '''
DELETE FROM Registro
WHERE ID_PIEZA = "salida" '''

cursor.execute(query)
pd.read_sql("SELECT * FROM Registro", conexion)

,FECHA,ID_PROVEEDOR,ID_PIEZA,CANTIDAD
0,2025-04-24,2,26,15
1,2025-04-24,100,101,97
